<a href="https://colab.research.google.com/github/iskra3138/UDA/blob/main/Office31_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Mar 21 00:58:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Downloading source and dataset

### source files

In [2]:
!git clone https://github.com/tim-learn/SHOT.git

fatal: destination path 'SHOT' already exists and is not an empty directory.


In [3]:
cd SHOT/object/

/content/SHOT/object


### office31 dataset

In [4]:
!gdown  https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE

Downloading...
From: https://drive.google.com/uc?id=0B4IapRTv9pJ1WGZVd1VDMmhwdlE
To: /content/SHOT/object/domain_adaptation_images.tar.gz
77.2MB [00:00, 246MB/s]


In [5]:
!tar -xvzf domain_adaptation_images.tar.gz

amazon/images/back_pack/frame_0001.jpg
amazon/images/back_pack/frame_0002.jpg
amazon/images/back_pack/frame_0003.jpg
amazon/images/back_pack/frame_0004.jpg
amazon/images/back_pack/frame_0005.jpg
amazon/images/back_pack/frame_0006.jpg
amazon/images/back_pack/frame_0007.jpg
amazon/images/back_pack/frame_0008.jpg
amazon/images/back_pack/frame_0009.jpg
amazon/images/back_pack/frame_0010.jpg
amazon/images/back_pack/frame_0011.jpg
amazon/images/back_pack/frame_0012.jpg
amazon/images/back_pack/frame_0013.jpg
amazon/images/back_pack/frame_0014.jpg
amazon/images/back_pack/frame_0015.jpg
amazon/images/back_pack/frame_0016.jpg
amazon/images/back_pack/frame_0017.jpg
amazon/images/back_pack/frame_0018.jpg
amazon/images/back_pack/frame_0019.jpg
amazon/images/back_pack/frame_0020.jpg
amazon/images/back_pack/frame_0021.jpg
amazon/images/back_pack/frame_0022.jpg
amazon/images/back_pack/frame_0023.jpg
amazon/images/back_pack/frame_0024.jpg
amazon/images/back_pack/frame_0025.jpg
amazon/images/back_pack/f

In [6]:
ll

total 75512
drwxr-xr-x 3 root     4096 Mar 21 00:04 amazon/
drwxr-xr-x 3 root     4096 Mar 21 00:04 data/
-rw-r--r-- 1 root     2797 Mar 21 00:04 data_list.py
-rw-r--r-- 1 root 77187046 Mar 21 00:59 domain_adaptation_images.tar.gz
drwxr-xr-x 3 root     4096 Mar 21 00:04 dslr/
-rw-r--r-- 1 root     9903 Mar 21 00:04 image_multisource.py
-rw-r--r-- 1 root    14024 Mar 21 00:04 image_multitarget.py
-rw-r--r-- 1 root    11369 Mar 21 00:04 image_pretrained.py
-rw-r--r-- 1 root    16232 Mar 21 00:04 image_source.py
-rw-r--r-- 1 root    16393 Mar 21 00:04 image_target_oda.py
-rw-r--r-- 1 root    15741 Mar 21 00:04 image_target.py
-rw-r--r-- 1 root     3361 Mar 21 00:04 loss.py
-rw-r--r-- 1 root     5193 Mar 21 00:04 network.py
drwxr-xr-x 2 root     4096 Mar 21 00:04 __pycache__/
-rw-r--r-- 1 root     3312 Mar 21 00:04 run.sh
drwxr-xr-x 3 root     4096 Mar 21 00:04 webcam/


# Making Data Pipeline

In [7]:
# Making files about paths of images
# ref: https://github.com/tim-learn/SHOT/issues/2 

import os, glob 
data_path = './data/office'
if not os.path.exists(data_path) :
  os.makedirs(data_path)

datasets = ['amazon', 'dslr', 'webcam']
classes = os.listdir('./amazon/images/')

for dataset in datasets:
  img_paths_file = open(os.path.join(data_path,'{}_list.txt'.format(dataset)), 'w')
  for i, cls in enumerate(classes): 
    for filename in glob.glob(os.path.join(dataset,'images',cls,'*.jpg')):
      img_paths_file.write("{} {}\n".format(filename,i))
  img_paths_file.close()

In [8]:
NUM_WORKERS = 2
BATCHSIZE = 64
folder = './data/'
names = ['amazon', 'dslr', 'webcam']
CLASS_NUM = 31
SOURCE = names[1]
TARGET = names[0]
TRTE = 'val'
DSET = 'office'
NET = 'resnet50'
MAX_EPOCH=100

NAME_SRC = SOURCE[0].upper()

S_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(SOURCE)
TEST_DSET_PATH = folder + DSET + '/' + '{}_list.txt'.format(TARGET)

In [9]:
import os
import torchvision
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from data_list import ImageList
import random

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

def image_train(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.RandomCrop(crop_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ])

def image_test(resize_size=256, crop_size=224, alexnet=False):
  if not alexnet:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                   std=[0.229, 0.224, 0.225])
  else:
    normalize = Normalize(meanfile='./ilsvrc_2012_mean.npy')
  return  transforms.Compose([
        transforms.Resize((resize_size, resize_size)),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
        normalize
    ])
  
def data_load(): 
    ## prepare data
    dsets = {}
    dset_loaders = {}
    train_bs = BATCHSIZE
    txt_src = open(S_DSET_PATH).readlines()
    txt_test = open(TEST_DSET_PATH).readlines()

    if TRTE == "val":
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        # print(dsize, tr_size, dsize - tr_size)
        tr_txt, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
    else:
        dsize = len(txt_src)
        tr_size = int(0.9*dsize)
        _, te_txt = torch.utils.data.random_split(txt_src, [tr_size, dsize - tr_size])
        tr_txt = txt_src

    dsets["source_tr"] = ImageList(tr_txt, transform=image_train())
    dset_loaders["source_tr"] = DataLoader(dsets["source_tr"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["source_te"] = ImageList(te_txt, transform=image_test())
    dset_loaders["source_te"] = DataLoader(dsets["source_te"], batch_size=train_bs, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)
    dsets["test"] = ImageList(txt_test, transform=image_test())
    dset_loaders["test"] = DataLoader(dsets["test"], batch_size=train_bs*2, shuffle=True, num_workers=NUM_WORKERS, drop_last=False)

    return dset_loaders


In [10]:
dset_loaders = data_load()

acc_init = 0
max_iter = MAX_EPOCH * len(dset_loaders["source_tr"])
interval_iter = max_iter // 10
iter_num = 0


# Training by PyTorch 

In [11]:
import torch.nn as nn
import torch.optim as optim
from loss import CrossEntropyLabelSmooth
import network, loss

In [12]:
def op_copy(optimizer):
    for param_group in optimizer.param_groups:
        param_group['lr0'] = param_group['lr']
    return optimizer

def lr_scheduler(optimizer, iter_num, max_iter, gamma=10, power=0.75):
    decay = (1 + gamma * iter_num / max_iter) ** (-power)
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr0'] * decay
        param_group['weight_decay'] = 1e-3
        param_group['momentum'] = 0.9
        param_group['nesterov'] = True
    return optimizer

def cal_acc(loader, netF, netB, netC, flag=False):
    start_test = True
    with torch.no_grad():
        iter_test = iter(loader)
        for i, _ in enumerate(loader):
        #for i in range(len(loader)):  
            data = iter_test.next()
            inputs = data[0]
            labels = data[1]
            inputs = inputs.cuda()
            outputs = netC(netB(netF(inputs)))
            if start_test:
                all_output = outputs.float().cpu()
                all_label = labels.float()
                start_test = False
            else:
                all_output = torch.cat((all_output, outputs.float().cpu()), 0)
                all_label = torch.cat((all_label, labels.float()), 0)

    all_output = nn.Softmax(dim=1)(all_output)
    _, predict = torch.max(all_output, 1)
    accuracy = torch.sum(torch.squeeze(predict).float() == all_label).item() / float(all_label.size()[0])
    mean_ent = torch.mean(loss.Entropy(all_output)).cpu().data.item()
   
    if flag:
        matrix = confusion_matrix(all_label, torch.squeeze(predict).float())
        acc = matrix.diagonal()/matrix.sum(axis=1) * 100
        aacc = acc.mean()
        aa = [str(np.round(i, 2)) for i in acc]
        acc = ' '.join(aa)
        return aacc, acc
    else:
        return accuracy*100, mean_ent

def cal_acc_oda(loader, netF, netB, netC):
    start_test = True
    with torch.no_grad():
        iter_test = iter(loader)
        for i in range(len(loader)):
            data = iter_test.next()
            inputs = data[0]
            labels = data[1]
            inputs = inputs.cuda()
            outputs = netC(netB(netF(inputs)))
            if start_test:
                all_output = outputs.float().cpu()
                all_label = labels.float()
                start_test = False
            else:
                all_output = torch.cat((all_output, outputs.float().cpu()), 0)
                all_label = torch.cat((all_label, labels.float()), 0)

    all_output = nn.Softmax(dim=1)(all_output)
    _, predict = torch.max(all_output, 1)
    ent = torch.sum(-all_output * torch.log(all_output + 1e-5), dim=1) / np.log(CLASS_NUM)
    ent = ent.float().cpu()
    initc = np.array([[0], [1]])
    kmeans = KMeans(n_clusters=2, random_state=0, init=initc, n_init=1).fit(ent.reshape(-1,1))
    threshold = (kmeans.cluster_centers_).mean()

    predict[ent>threshold] = CLASS_NUM
    matrix = confusion_matrix(all_label, torch.squeeze(predict).float())
    matrix = matrix[np.unique(all_label).astype(int),:]

    acc = matrix.diagonal()/matrix.sum(axis=1) * 100
    unknown_acc = acc[-1:].item()

    return np.mean(acc[:-1]), np.mean(acc), unknown_acc
    # return np.mean(acc), np.mean(acc[:-1])

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torch.autograd import Variable
import math
import torch.nn.utils.weight_norm as weightNorm
from collections import OrderedDict

def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)


res_dict = {"resnet18":models.resnet18, "resnet34":models.resnet34, "resnet50":models.resnet50, 
"resnet101":models.resnet101, "resnet152":models.resnet152, "resnext50":models.resnext50_32x4d, "resnext101":models.resnext101_32x8d}
class ResBase(nn.Module):
    def __init__(self, res_name):
        super(ResBase, self).__init__()
        model_resnet = res_dict[res_name](pretrained=True)
        self.conv1 = model_resnet.conv1
        self.bn1 = model_resnet.bn1
        self.relu = model_resnet.relu
        self.maxpool = model_resnet.maxpool
        self.layer1 = model_resnet.layer1
        self.layer2 = model_resnet.layer2
        self.layer3 = model_resnet.layer3
        self.layer4 = model_resnet.layer4
        self.avgpool = model_resnet.avgpool
        self.in_features = model_resnet.fc.in_features

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x

class feat_bootleneck(nn.Module):
    def __init__(self, feature_dim, bottleneck_dim=256, type="ori"):
        super(feat_bootleneck, self).__init__()
        self.bn = nn.BatchNorm1d(bottleneck_dim, affine=True)
        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(p=0.5)
        self.bottleneck = nn.Linear(feature_dim, bottleneck_dim)
        self.bottleneck.apply(init_weights)
        self.type = type

    def forward(self, x):
        x = self.bottleneck(x)
        if self.type == "bn":
            x = self.bn(x)
        return x

class feat_classifier(nn.Module):
    def __init__(self, class_num, bottleneck_dim=256, type="linear"):
        super(feat_classifier, self).__init__()
        self.type = type
        if type == 'wn':
            self.fc = weightNorm(nn.Linear(bottleneck_dim, class_num), name="weight")
            self.fc.apply(init_weights)
        else:
            self.fc = nn.Linear(bottleneck_dim, class_num)
            self.fc.apply(init_weights)

    def forward(self, x):
        x = self.fc(x)
        return x

In [13]:
# Network Define 
netF = ResBase(res_name=NET).cuda()

netB = feat_bootleneck(type="bn", feature_dim=netF.in_features, bottleneck_dim=256).cuda()
netC = feat_classifier(type="wn", class_num = 31, bottleneck_dim=256).cuda()

param_group = []
learning_rate = 0.01
for k, v in netF.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate*0.1}]
for k, v in netB.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate}]
for k, v in netC.named_parameters():
    param_group += [{'params': v, 'lr': learning_rate}]   
optimizer = optim.SGD(param_group)
optimizer = op_copy(optimizer)

netF.train()
netB.train()
netC.train()

feat_classifier(
  (fc): Linear(in_features=256, out_features=31, bias=True)
)

In [31]:
# Train

iter_num = 0
while iter_num < max_iter: 
    iter_source = iter(dset_loaders["source_tr"])
    inputs_source, labels_source = iter_source.next()

    if inputs_source.size(0) == 1:
        continue

    iter_num += 1
    lr_scheduler(optimizer, iter_num=iter_num, max_iter=max_iter)

    inputs_source, labels_source = inputs_source.cuda(), labels_source.cuda()
    outputs_source = netC(netB(netF(inputs_source)))
    classifier_loss = CrossEntropyLabelSmooth(num_classes=31, epsilon=0.1)(outputs_source, labels_source) ## 다른부분           
    
    optimizer.zero_grad()
    classifier_loss.backward()
    optimizer.step()

    if iter_num % interval_iter == 0 or iter_num == max_iter:
        netF.eval()
        netB.eval()
        netC.eval()

        lrs = []
        for param_group in optimizer.param_groups:
          lrs.append(param_group['lr'])

        if DSET=='VISDA-C':
            acc_s_te, acc_list = cal_acc(dset_loaders['source_te'], netF, netB, netC, True)
            log_str = 'Task: {}, Iter:{}/{}; Accuracy = {:.2f}%'.format(NAME_SRC, iter_num, max_iter, acc_s_te) + '\n' + acc_list
        else:
            acc_s_te, _ = cal_acc(dset_loaders['test'], netF, netB, netC, False)
            log_str = 'Task: {}, Iter:{}/{}; lr1:{:.5f}; lr2:{:.5f}; Accuracy = {:.2f}%'.format(NAME_SRC, 
                                                                                                iter_num, 
                                                                                                max_iter,
                                                                                                lrs[0],
                                                                                                lrs[-1], 
                                                                                                acc_s_te)
        #args.out_file.write(log_str + '\n')
        #args.out_file.flush()
        print(log_str+'\n')
        '''
        if acc_s_te >= acc_init:
            acc_init = acc_s_te
            best_netF = netF.state_dict()
            best_netB = netB.state_dict()
            best_netC = netC.state_dict()
        '''
        netF.train()
        netB.train()
        netC.train()

Task: D, Iter:70/700; lr1:0.00059; lr2:0.00595; Accuracy = 57.58%

Task: D, Iter:140/700; lr1:0.00044; lr2:0.00439; Accuracy = 58.64%

Task: D, Iter:210/700; lr1:0.00035; lr2:0.00354; Accuracy = 58.43%

Task: D, Iter:280/700; lr1:0.00030; lr2:0.00299; Accuracy = 58.43%

Task: D, Iter:350/700; lr1:0.00026; lr2:0.00261; Accuracy = 58.47%

Task: D, Iter:420/700; lr1:0.00023; lr2:0.00232; Accuracy = 58.71%

Task: D, Iter:490/700; lr1:0.00021; lr2:0.00210; Accuracy = 58.40%

Task: D, Iter:560/700; lr1:0.00019; lr2:0.00192; Accuracy = 59.32%

Task: D, Iter:630/700; lr1:0.00018; lr2:0.00178; Accuracy = 59.42%

Task: D, Iter:700/700; lr1:0.00017; lr2:0.00166; Accuracy = 59.14%

